# Hyperparameter Tuning using HyperDrive

In [1]:
from azureml.core.workspace import Workspace
from azureml.core import Experiment, Model, Webservice
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
import joblib

import data_prep

## Dataset

In [2]:
ws = Workspace.from_config()
ws.write_config(path='./training')
experiment_name = 'ddos_hyperdrive_experiment'

experiment=Experiment(ws, experiment_name)

dataset = data_prep.get_DDoS_dataset(ws)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FZXYV3EST to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/feff3db9-0096-4330-a147-f828aff9ae83/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


WARNING - Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


## Find or Create Compute Cluster

In [3]:
cpu_cluster_name = "cpu-cluster"

# Check if the cluster exists. If there is an error, create the cluster
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2',
                                                            max_nodes=10)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [18]:
# Early termination policy
early_termination_policy = BanditPolicy(slack_amount=0.2)

# Random Parameter sampling
param_sampling = RandomParameterSampling({
                    "n_estimators": choice(10,50,100,150,200),
                    "max_depth": choice(0,2,5,10),
                    "min_samples_split": choice(2,3,4,5),
                    "min_samples_leaf": choice(1,2,3,4,5)
                })

# SKLearn Estimator
estimator = SKLearn(source_directory="./training", 
                entry_script="train.py",
                compute_target=cpu_cluster)

# Hyperdrive Configuration
hyperdrive_run_config = HyperDriveConfig(param_sampling,
                                    primary_metric_name="F1", 
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=50,
                                    estimator=estimator,
                                    policy=early_termination_policy)

In [19]:
hyperdrive_run = experiment.submit(hyperdrive_run_config)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [20]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [12]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Num Estimators:',parameter_values[7])
print('\n Max Depth:',parameter_values[1])
print('\n Min Samples Split:',parameter_values[5])
print('\n Min Samples Leaf:',parameter_values[3])

Best Run Id:  HD_38bd153e-11cd-44a6-9f65-1a33a8627178_2

 Accuracy: 1.0

 Num Estimators: 50

 Max Depth: 0

 Min Samples Split: 3

 Min Samples Leaf: 3


In [13]:
model = best_run.register_model(model_name = 'best_hyperdrive_model', model_path = 'outputs/model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service